In [14]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import json
import time

In [8]:
# Load your Excel
df = pd.read_excel(r"D:\Coding\Air Crash Investigation\data\raw\Location_Data.xlsx")


In [15]:
# Initialize geolocator
geolocator = Nominatim(user_agent="geoapi-excel")

# Set up the RateLimiter with better delay and retries
geocode = RateLimiter(
    geolocator.geocode,
    min_delay_seconds=2,
    max_retries=3,
    error_wait_seconds=10,
    swallow_exceptions=True  # Prevent crashes
)

In [16]:
# Get the unique locations (drop NaN and strip whitespace)
unique_locations = df['Location'].dropna().drop_duplicates().str.strip()

# Load previous partial results if any
try:
    with open('location_country_map_partial.json', 'r') as f:
        location_country_map = json.load(f)
    print(f"Loaded {len(location_country_map)} previously saved locations.")
except FileNotFoundError:
    location_country_map = {}


In [ ]:
# Loop through unique locations
for idx, loc in enumerate(unique_locations, start=1):
    if loc in location_country_map:
        print(f"[{idx}] Skipping (already processed): {loc}")
        continue

    if not loc:
        location_country_map[loc] = None
        print(f"[{idx}] Empty location, skipped.")
        continue

    print(f"[{idx}] Geocoding: {loc}")
    location = geocode(loc, timeout=10)

    if location:
        country = location.address.split(',')[-1].strip()
        print(f" --> Found country: {country}")
    else:
        country = None
        print(" --> No result found.")

    location_country_map[loc] = country

    # Save progress every 100 locations
    if idx % 100 == 0:
        with open('location_country_map_partial.json', 'w') as f:
            json.dump(location_country_map, f)
        print(f"Saved progress at {idx} locations.")

# Save final results
with open('location_country_map_final.json', 'w') as f:
    json.dump(location_country_map, f)

print("Geocoding completed and saved to 'location_country_map_final.json'.")

[1] Geocoding: Verona
 --> Found country: Italia
[2] Geocoding: off Felixtowe RNAS
 --> No result found.
[3] Geocoding: Acadia Siding, Cape Province
 --> No result found.
[4] Geocoding: 10 km N of El Shereik
 --> No result found.
[5] Geocoding: Östanå
 --> Found country: Sverige
[6] Geocoding: Golders Green
 --> Found country: United Kingdom
[7] Geocoding: Havana harbour
 --> Found country: United Kingdom
[8] Geocoding: Meerut
 --> Found country: India
[9] Geocoding: Lago Maggiore
 --> Found country: Italia
[10] Geocoding: 5,6 km off Calais [The English Channel]
 --> No result found.
[11] Geocoding: Evere Airfield
 --> No result found.
[12] Geocoding: Berck-sur-Mer
 --> Found country: France
[13] Geocoding: near Senlis
 --> No result found.
[14] Geocoding: Baoding, Hebei Province
 --> Found country: 中国
[15] Geocoding: Thieuloy-Saint-Antoine
 --> Found country: France
[16] Geocoding: 32 km N off Havana, Cuba
 --> No result found.
[17] Geocoding: Monsures, Somme
 --> Found country: Franc

RateLimiter caught an error, retrying (0/3 tries). Called with (*('1,5 km SW of An Loc',), **{'timeout': 10}).
Traceback (most recent call last):
  File "d:\Coding\Air Crash Investigation\aci\lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
  File "d:\Coding\Air Crash Investigation\aci\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "d:\Coding\Air Crash Investigation\aci\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\Coding\Air Crash Investigation\aci\lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
  File "d:\Coding\Air Crash Investigation\aci\lib\site-packages\urllib3\connectionpool.py", line 488, in _make_request
    raise new_e
  File "

 --> No result found.
[8778] Geocoding: 8 km from An Loc
 --> No result found.
[8779] Geocoding: 68 km SE of Paramaribo-Zanderij International Airport (PBM)
 --> No result found.
[8780] Geocoding: Sofia
 --> Found country: България
[8781] Geocoding: 1,5 km NE of An Loc
 --> No result found.
[8782] Geocoding: 1,2 km from Bratsk Airport (BTK)
 --> No result found.
[8783] Geocoding: Poltava Region
 --> Found country: Sverige
[8784] Geocoding: 5 km SW of Palermo
 --> No result found.
[8785] Geocoding: 0,9 km from Valledupar Airport (VUP)
 --> No result found.
[8786] Geocoding: Argyz district, Tatarstan
 --> No result found.
[8787] Geocoding: ca 150 km W of Ciudad Bolívar
 --> No result found.
[8788] Geocoding: El Obeid Airport (EBD)
 --> Found country: السودان
[8789] Geocoding: near Kirensk
 --> No result found.
[8790] Geocoding: Okeechobee, FL
 --> Found country: United States
[8791] Geocoding: Tokyo-Haneda International Airport
 --> Found country: 日本
[8792] Geocoding: Svetlogorsk
 --> Fo

KeyboardInterrupt: 

In [19]:
# Map the country back using the saved results
df['Country'] = df['Location'].map(location_country_map)

# Save the updated Excel
output_file = r"D:\Coding\Air Crash Investigation\data\processed\locations_with_countries.xlsx"
df.to_excel(output_file, index=False)
print(f"Excel saved to: {output_file}")


Excel saved to: D:\Coding\Air Crash Investigation\data\processed\locations_with_countries.xlsx
